In [12]:
import os
import pandas as pd

directory = os.path.expanduser('./Caudal_Medio_Dia_Historico/')
directory_norm = os.path.expanduser('./Caudal_Medio_Dia_Historico_Normalizado/')

estaciones = {}
estaciones_norm = {}

def process_directory(dir_path, data_dict):
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            path = os.path.join(dir_path, filename)
            var_name = os.path.splitext(filename)[0].replace(' ', '_').lower()
            df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')
            data_dict[var_name] = df
            print(f"{var_name}")

process_directory(directory, estaciones)
process_directory(directory_norm, estaciones_norm)


la_esperanza_nechi
la_raya
las_varas
magangue
sucre
tres_cruces
villa_teresa
las_varas_norm
la_esperanza_nechi_norm
la_raya_norm
magangue_norm
sucre_norm
tres_cruces_norm
villa_teresa_norm


In [13]:
def transform_data(data, lat, lon, station_name):
    melted_data = data.melt(id_vars=["year", "month"], var_name="day", value_name="Caudal")
    melted_data.dropna(subset=["Caudal"], inplace=True)
    melted_data['day'] = melted_data['day'].astype(int)
    melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']]) + pd.to_timedelta('00:00:00')
    melted_data.sort_values('Fecha', inplace=True)
    melted_data['Latitud'] = lat
    melted_data['Longitud'] = lon
    melted_data['Estacion'] = station_name.replace('_', ' ').title()
    return melted_data

coords = {
    "la_esperanza_nechi": (8.030111111, -74.78555556),
    "la_raya": (8.344555556, -74.56122222),
    "las_varas": (8.39, -74.56),
    "magangue": (9.24, -74.74166667),
    "sucre": (8.809722222, -74.72222222),
    "tres_cruces": (8.703083333, -74.51730556),
    "villa_teresa": (9.023972222, -74.97666667)
}

In [14]:
transformed_data = {}
for name, df in estaciones.items():
    lat, lon = coords[name]
    station_name = name
    transformed_df = transform_data(df, lat, lon, station_name)
    transformed_data[name] = transformed_df

for name, norm_df in estaciones_norm.items():
    base_name = name.replace('_norm', '')
    if base_name in transformed_data:
        station_name = base_name
        norm_df_melted = transform_data(norm_df, None, None, station_name)
        norm_df_melted = norm_df_melted[['Fecha', 'Caudal']].rename(columns={'Caudal': 'Caudal_norm'})
        transformed_data[base_name] = pd.merge(transformed_data[base_name], norm_df_melted, on='Fecha', how='left')


In [15]:
for name, df in transformed_data.items():
    df['Caudal'] = df['Caudal'].astype(str).str.replace(',', '.').astype(float)
    if 'Caudal_norm' in df.columns:
        df['Caudal_norm'] = df['Caudal_norm'].astype(str).str.replace(',', '.').astype(float)

for name, df in transformed_data.items():
    df[name] = df.apply(lambda row: row['Caudal'] if row['Estacion'].replace(' ', '_').lower() == name else 0, axis=1)


In [16]:
combined_data = pd.concat(transformed_data.values(), ignore_index=True)
properties = ['Fecha', 'Caudal', 'Caudal_norm', 'Estacion'] + list(transformed_data.keys())


In [17]:
def clean_nan_values(df):
    for col in df.columns:
        df[col].fillna(0, inplace=True)
    return df

combined_data = clean_nan_values(combined_data)

In [18]:
def df_to_geojson(df, properties, lat='Latitud', lon='Longitud'):
    geojson = {'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Point',
                'coordinates': [row[lon], row[lat]]
            }
        }
        for prop in properties:
            if isinstance(row[prop], pd.Timestamp):
                feature['properties'][prop] = row[prop].isoformat()
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson



In [19]:

import json

geojson_data = df_to_geojson(combined_data, properties)
geojson_path = os.path.expanduser('./Caudales_norm_etiquetas_ajustado.geojson')
with open(geojson_path, 'w') as f:
    json.dump(geojson_data, f)

print(f"GeoJSON file created at {geojson_path}")

GeoJSON file created at ./Caudales_norm_etiquetas_ajustado.geojson
